<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"></ul></div>

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import mode
ZERO_ADJ_COL = ['imu_ax', 'imu_ay', 'imu_az', 'imu_gx', 'imu_gy', 'imu_gz']

In [2]:
"""
df: the raw dataframe you want to process

return: preprocessed df, resample by 0.01S and calibrate by mode(the most frequency appeared)
"""
def __build_play_df(df, calibrate=True, resample=True):
    # crop desired play time interval
    play_df = df.copy()
    play_df.dropna(inplace=True)
    play_df[play_df == 0] = np.nan
    # resample for more samples
    if resample:
        play_df.loc[:, 'timestamp'] = pd.to_datetime(
            play_df['timestamp'], unit='s')
        play_df.loc[:, 'timestamp'] = play_df['timestamp'].apply(
            lambda x: x.tz_localize('UTC').tz_convert('Asia/Taipei'))
        play_df = play_df.set_index('timestamp').resample(
            '0.01S').mean()
        play_df = play_df.interpolate(method='linear')
        play_df.reset_index(inplace=True)
        play_df.loc[:, 'timestamp'] = play_df['timestamp'].apply(
            lambda x: x.timestamp())
        play_df.fillna(method='ffill', inplace=True)

    # implement zero adjust for needed columns
    if calibrate:
        modes_dict = {}
        copy_df = play_df.copy()
        for col in ZERO_ADJ_COL:
            mode_ = mode(copy_df[col])[0]
            modes_dict[col] = mode_

        # only considered attributes need zero adjust
        for col in ZERO_ADJ_COL:
            copy_df.loc[:, col] = copy_df[col] - modes_dict[col]

        play_df = copy_df

    return play_df

In [3]:
ddf = pd.read_csv('../data/bb_left_forearm_csv/L_2018-09-22_141420.csv')
play_df = __build_play_df(ddf)
play_df.describe()

,timestamp,imu_ax,imu_ay,imu_az,imu_gx,imu_gy,imu_gz
count,5.297000e+03,5297.000000,5297.000000,5297.000000,5297.000000,5297.000000,5297.000000
mean,1.537597e+09,0.052092,-0.041186,1.532541,2.329466,1.837803,1.190681
std,1.529257e+01,2.009217,2.387346,1.821444,58.964106,45.397879,72.086120
min,1.537597e+09,-12.156525,-18.862500,-3.987525,-122.625750,-171.051000,-183.166525
25%,1.537597e+09,-0.981620,-0.394425,0.356260,-35.919200,-11.291520,-46.069340
50%,1.537597e+09,0.029300,0.262150,1.266525,1.602150,1.879860,0.085460
75%,1.537597e+09,0.940300,1.099880,2.494260,16.107170,15.319800,20.544440
max,1.537597e+09,12.899500,4.908620,11.319340,194.647190,260.668960,261.215200
